# Rysk Finance

Defi options AMM

[Docs](shttps://docs.rysk.finance/getting-started/what-is-rysk)

Rysk options are priced using a AMM.


# Trading competition

[Trading comp](https://comp.rysk.finance/) is open until June.

# Data retrieval

We can retrieve data from the Rysk finance contracts from the follwoing [subgraph](https://api.goldsky.com/api/public/project_clhf7zaco0n9j490ce421agn4/subgraphs/devey/0.0.2/gn)

There is a depth of on chain data from the subgraph.

For example, we can retrieve all of the available markets as so;

In [117]:
# install dependencies
!pip install pandas ccxt web3

You should consider upgrading via the '/home/tom/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [91]:
# we can retrieve the data using a simple client;
from dataclasses import dataclass
import requests
import json

# -H "Content-Type: application/json" -X POST -d 
@dataclass
class SubgraphClient:
    url: str
    
    
    def query(self, query):
        """Simple function to call a subgraph query."""
        headers = {
            "Content-Type": "application/json"
        }
        subgraph_query = {
            "query": query
        }
        response = requests.post(
            url=self.url,
            headers=headers,
            data=json.dumps(subgraph_query)
        )
        data = json.loads(response.content)['data']
        return data

In [92]:
subgraph_url = "https://api.goldsky.com/api/public/project_clhf7zaco0n9j490ce421agn4/subgraphs/devey/0.0.2/gn"

subgraph_query = """
{series 
  {   
    id 
    expiration 
    netDHVExposure 
    strike
    isPut
    isBuyable
    isSellable
  }
}
"""

client = SubgraphClient(subgraph_url)
 
results = client.query(subgraph_query)

results['series'][:1]



[{'id': '0x01f460be7389b109cc3599941166ea851d0b7c787badf04b1f276d3ce9269a34',
  'expiration': '1688112000',
  'netDHVExposure': '-368000000000000000000',
  'strike': '1700000000000000000000',
  'isPut': True,
  'isBuyable': True,
  'isSellable': True}]

In [93]:
# we then parse into a pandas dataframe.

import pandas as pd
from datetime import datetime

# we need to convert the timestamp of the expiration to a date, we also need to clean up the decimals.
# conveniently the options expirations align with those from deribit;
# we format to allow easier look up later


price_devisor = 1_000_000_000_000_000_000
exposure_devisor = 100_000_000_000_000_0000

def from_timestamp(date_string):
    """Parse a timestamp."""
    return datetime.fromtimestamp(int(date_string))

def to_human_format(row):
    """
    Format the row to align to the ccxt unified client.
    'ETH-16MAY23-1550-C'
    """

    month_code = row.expiration_datetime.strftime("%b").upper()
    day = row.expiration_datetime.strftime("%d")
    year = str(row.expiration_datetime.year)[2:]
    strike_price = str(row.strike_price)

    return f"ETH-{day}{month_code}{year}-{strike_price}-{'P' if row.isPut else 'C'}"


df = pd.DataFrame(results['series'])


# We format the data
df['expiration_datetime'] = df["expiration"].apply(from_timestamp)
df['strike_price'] = df['strike'].apply(lambda x: int(int(x) / price_devisor))
df['net_DHV_exposure'] = df['netDHVExposure'].apply(lambda x: int(int(x) / exposure_devisor))
df["human_strike"] = df.apply(to_human_format, axis=1)




# we filter out all markets that have already closed;
df = df[df["expiration_datetime"] > datetime.now()]



# we filter out markets that we cant trade at all
df = df[df["isBuyable"] | df['isSellable']]


# We only display our interested columns
columns = ["human_strike", "expiration_datetime", "strike_price", "net_DHV_exposure", "isPut", "isBuyable", "isSellable"]
df[columns].sort_values(["expiration_datetime", "strike_price"]).head(50)


,human_strike,expiration_datetime,strike_price,net_DHV_exposure,isPut,isBuyable,isSellable
26,ETH-19MAY23-1750-P,2023-05-19 10:00:00,1750,-577,True,True,True
78,ETH-19MAY23-1800-C,2023-05-19 10:00:00,1800,-367,False,True,True
79,ETH-19MAY23-1800-P,2023-05-19 10:00:00,1800,-432,True,True,True
43,ETH-19MAY23-1850-C,2023-05-19 10:00:00,1850,-398,False,True,True
82,ETH-19MAY23-1850-P,2023-05-19 10:00:00,1850,-176,True,True,True
72,ETH-19MAY23-1900-C,2023-05-19 10:00:00,1900,-487,False,True,True
41,ETH-26MAY23-1650-P,2023-05-26 10:00:00,1650,-540,True,True,True
30,ETH-26MAY23-1700-P,2023-05-26 10:00:00,1700,-530,True,True,True
40,ETH-26MAY23-1750-P,2023-05-26 10:00:00,1750,-384,True,True,True
3,ETH-26MAY23-1800-C,2023-05-26 10:00:00,1800,-355,False,True,True


# Exposure Arbitrages

Conceptually, the DHV is pricing its options based on demand, where by the price is a function of the expected volitity based on the BlackSholes formula combined with net exposure at each strike.

We can therefore seek to find a number of arbitrages within the markets;

We will initially demonstrate a function to identify arbitrage based on short term options versus long term options.

As a basic strategy, we will look at each side of each of the markets. For each of the tradable options, we will look for circumstances whereby the option may be immediately sold to either capitalise on the differential between the price, or the differentical between the time.


In [94]:
# For each market, we will determine
print(df.head())

print(df[columns].sort_values(["expiration_datetime", "strike_price"]).head())


                                                   id  expiration   
0   0x01f460be7389b109cc3599941166ea851d0b7c787bad...  1688112000  \
2   0x04469230a59e8753063773df8102976628002e4dae93...  1688112000   
3   0x05918f06eec70037579cac539edab276c3adda716d77...  1685088000   
19  0x2ae92346e1d10998372d1a93e925148c8b4d701042cf...  1688112000   
20  0x2b14a6a6ef49e48f440482ec895fbc37ae9c2f0e8e2f...  1685088000   

            netDHVExposure                  strike  isPut  isBuyable   
0   -368000000000000000000  1700000000000000000000   True       True  \
2   -247000000000000000000  1800000000000000000000  False       True   
3   -355310000000000000000  1800000000000000000000  False       True   
19  -610500000000000000000  2200000000000000000000  False       True   
20  -265000000000000000000  1850000000000000000000   True       True   

    isSellable expiration_datetime  strike_price  net_DHV_exposure   
0         True 2023-06-30 10:00:00          1700              -368  \
2         Tr

In [95]:
def filter_candidates_buy_type(row)

SyntaxError: expected ':' (822498097.py, line 1)

In [96]:
def does_arb_exist(row, display=False):
    """We will check the inital dataframe."""
    # find similar candidates
    candidates = df[df.isPut == row.isPut]
    
    # we filter out older markets
    candidates = candidates[candidates.expiration_datetime >= row.expiration_datetime]
    
    if row.isBuyable:
        candidates = candidates[candidates.isSellable]
    
    if row.isSellable:
        candidates = candidates[candidates.isBuyable]
    
    if row.isPut:
        # we want to look for puts which have a strike greater than our current row.
        candidates = candidates[candidates.strike >= row.strike]
        if row.isBuyable:
            # we are looking to *buy* cheap puts then sell them on.
            # expensive options are those where the dhv has has sold a lot at that strike 
            # and is therefore increasing the price to compensate. 
            # This is denoted with a negative exposure
            
            # our candidates will be those with new_dhv_exposure > our row 
            candidates = candidates[candidates.net_DHV_exposure > row.net_DHV_exposure]
            candidates.sort_values("net_DHV_exposure")
            
        else:
            # We now are looking to *sell* expensive puts 
            # and then buy cheap ones
            candidates = candidates[candidates.net_DHV_exposure < row.net_DHV_exposure]
            
    
    else:
        # we want to look for call which have a strike lower than our current row
        candidates = candidates[candidates.strike <= row.strike]
        if row.isBuyable:
            # we are looking to *buy* cheap puts then sell them on.
            # expensive puts are those where the dhv has has sold a lot at that strike 
            # and is therefore increasing the price to compensate. 
            # This is denoted with a negative exposure
            
            # our candidates will be those with new_dhv_exposure > our row 
            candidates = candidates[candidates.net_DHV_exposure > row.net_DHV_exposure]
        
        else:
            # We now are looking to *sell* expensive puts 
            # and then buy cheap ones
            candidates = candidates[candidates.net_DHV_exposure < row.net_DHV_exposure]
        
    def _format_row(row):
        """Format the row."""
        return f"{row.human_strike} @ {row.net_DHV_exposure}"
    

        
        
    if len(candidates):
        
#       # we now look for the biggest differences between the candidates and the row. 
        # we return this so we further can sort.
        
        candidates['value'] = abs(candidates.net_DHV_exposure - row.net_DHV_exposure)
        
        candidates = candidates.sort_values('value')
        if display:
            print(f"Potential Arb between: {_format_row(row)} & {_format_row(candidates.iloc[0])}")


    return False


df['arbs'] = df.apply(does_arb_exist, axis=1)
top_arbs = df.sort_values("arbs", ascending=False).head()


top_arbs.apply(does_arb_exist, axis=1, args=(True,))

Potential Arb between: ETH-30JUN23-1700-P @ -368 & ETH-30JUN23-1800-P @ -289
Potential Arb between: ETH-30JUN23-1800-C @ -247 & ETH-30JUN23-1700-C @ -155
Potential Arb between: ETH-19MAY23-1850-P @ -176 & ETH-30JUN23-2000-P @ -169
Potential Arb between: ETH-19MAY23-1800-P @ -432 & ETH-26MAY23-1800-P @ -337
Potential Arb between: ETH-19MAY23-1800-C @ -367 & ETH-26MAY23-1800-C @ -355


0     False
2     False
82    False
79    False
78    False
dtype: bool

# Price Retrieval 

Prices are extracted from the [Beyond Pricer](https://goerli.arbiscan.io/address/0xc939df369C0Fc240C975A6dEEEE77d87bCFaC259#readContract)

This allows us to take the id retrieved from the subgraph and use it to extract all of the prices, (Unfortunately this is iteratively, however this could be improved by implementing and deploying a smart contract which would collect the data in one call)

In [115]:
# we first need to import the contracts abi
# follow the instructions in ./third_party in order to compile contracts

with open("./third_party/dynamic-hedging/packages/contracts/artifacts/contracts/BeyondPricer.sol/BeyondPricer.json", "r") as file:
    abi = json.loads(file.read())['abi']
    
    

In [133]:
from web3 import Web3


w3 = Web3(Web3.HTTPProvider('https://arbitrum-goerli.rpc.thirdweb.com'))


address = '0xc939df369C0Fc240C975A6dEEEE77d87bCFaC259'
contract_instance = w3.eth.contract(address=address, abi=abi)

# to prove we have the correct contract
contract_instance.functions.collateralAsset().call()


'0x408c5755b5c7a0a28D851558eA3636CfC5b5b19d'

In [134]:
df.iloc[0]

id                     0x01f460be7389b109cc3599941166ea851d0b7c787bad...
expiration                                                    1688112000
netDHVExposure                                    -378000000000000000000
strike                                            1700000000000000000000
isPut                                                               True
isBuyable                                                           True
isSellable                                                          True
expiration_datetime                                  2023-06-30 10:00:00
strike_price                                                        1700
net_DHV_exposure                                                     170
human_strike                                          ETH-30JUN23-1700-C
Name: 0, dtype: object

In [194]:
# now we have our contract, we can try to retrieve the prices. 
# Thanks @Yassim for the support here.
from enum import Enum 

class Collateral(Enum):
    eth = "0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d"
    usdc = "unknown"
    
    @classmethod
    def is_supported(cls, collateral):
        if collateral not in cls.__members__:
            return False
        return True

def from_market_to_id(market):
    """We perform a lookup in our market data to return the onchain representation of the market"""
    is_put = market[-1] == "P"
    expiration_date_ts = to_timestamp(market)


def to_timestamp(human_format):
    """
    Convert the human format to a timestamp.
    Example input: "ETH-19MAY23-1800-C"
    Example output: 1688112000
    """

    date_str = human_format.split("-")[1]  # Extract the date part from the human format
    datetime_obj = datetime.strptime(date_str, "%d%b%y")  # Convert the date string to a datetime object
    timestamp = datetime_obj.timestamp()  # Get the timestamp in seconds since the epoch

    return int(timestamp)


def filter_by_human_format(df, human_format):
    """
    Filter the DataFrame based on the human-readable format and return a single row.
    """

    filtered_df = df[df["human_strike"] == human_format]
    if len(filtered_df) > 0:
        return filtered_df.iloc[0]  # Return the first row if found
    else:
        return None  # Return None if no matching row is found

    

def get_options_prices(market, amount=1, side="buy", collateral="eth"):
    """, 
    We call the beyond pricer to determine the prices for a market
    
    """
    if side not in ["buy", "sell"]:
        raise ValueError("Side must be buy or sell")
    
    filtered_row = filter_by_human_format(df, market)
    if filtered_row is None:
        raise TypeError("Unable to find the market")

        
    if not Collateral.is_supported(collateral):
        raise TypeError(f"Collateral {collateral} is not supported")
    # here we call the contract functions
    
    
    is_put = True if market[-1] == "P" else False
    args = (
            Collateral.eth.value,
            to_timestamp(market),
            is_put,
            str(filtered_row.strike_price),
            Collateral.eth.value,
            "0x3b3a1de07439eeb04492fa64a889ee25a130cdd3",
        )
    
    print(args)
    result = contract_instance.functions.quoteOptionPrice(args,
                                                          int(amount),
        True if side == "sell" else False,
        int(filtered_row.net_DHV_exposure))

    return result
        
market = "ETH-19MAY23-1800-C"

get_options_prices(market, collateral="eth")


('0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d', 1684447200, False, '1800', '0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d', '0x3b3a1de07439eeb04492fa64a889ee25a130cdd3')


ValidationError: 
Could not identify the intended function with name `quoteOptionPrice`, positional argument(s) of type `(<class 'tuple'>, <class 'int'>, <class 'bool'>, <class 'int'>)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `quoteOptionPrice`: ['quoteOptionPrice(tuple,uint256,bool,int256)']
Function invocation failed due to no matching argument types.

In [195]:


is_put = False
amount=20000000000000

filtered_row = filter_by_human_format(df, market)
    
args = [
    (
        Collateral.eth.value,
        to_timestamp(market),
        is_put,
        str(filtered_row.strike_price),
        Collateral.eth.value,
        "0x3b3a1de07439eeb04492fa64a889ee25a130cdd3",
    ),
    int(amount),
    True if side == "sell" else False,
    int(filtered_row.net_DHV_exposure),
]

result = contract_instance.functions.quoteOptionPrice(*args)
                                                      

ValidationError: 
Could not identify the intended function with name `quoteOptionPrice`, positional argument(s) of type `(<class 'tuple'>, <class 'int'>, <class 'bool'>, <class 'int'>)` and keyword argument(s) of type `{}`.
Found 1 function(s) with the name `quoteOptionPrice`: ['quoteOptionPrice(tuple,uint256,bool,int256)']
Function invocation failed due to no matching argument types.

In [120]:
# Unified crypto client.

import ccxt
# we filter out everything except ethereum options

exchange = ccxt.deribit()
markets = exchange.fetch_markets()

[f['id'] for f in markets if f['option'] and f['base'] == "ETH"][0]

'ETH-16MAY23-1550-C'